# STA 141C Homework 3

In [157]:
import numpy as np
import pandas as pd
import scipy.linalg as sl
from sklearn.linear_model import LinearRegression
from scipy.linalg import cho_factor, cho_solve
import matplotlib.pylab as plt
import scipy.sparse as sparse
import time
import math
from numpy.linalg import svd, eig
import numpy.linalg

In [158]:
# Read data of csv file. 
A = pd.read_csv("A.txt", sep=',', header=None)
Udf = pd.read_csv("U.txt", header=None)

# Udf.iloc[68]
# Udf.iloc[1]
# Udf.iloc[149]
# Udf.iloc[322]


# delete urls that are not in the right format (1, 68, 149, 322)
Udf.drop([1, 68, 149, 322], inplace=True)
A.drop([1, 68, 149, 322], axis=0, inplace=True)
A.drop([1, 68, 149, 322], axis=1, inplace=True)

In [159]:
A

,0,2,3,4,5,6,7,8,9,10,...,490,491,492,493,494,495,496,497,498,499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [160]:
# data preparing

Att = A.T
Att = np.array(Att)

d = 0.85
ri = Att.sum(axis=1)

H = np.zeros((496,496))

for i in range(len(ri)):
    for j in range(len(ri)):
        if ri[i] > 0:
            H[i][j] = d*(Att[i][j]/ri[i]) + (1-d)/496
        else:
            H[i][j] = 1/ 496



In [161]:
# data preparing

Att = A.T
Att = np.array(Att)

d = 0.85
ri = Att.sum(axis=1)

H = np.zeros((496,496))

for i in range(len(ri)):
    for j in range(len(ri)):
        if ri[i] > 0:
            H[i][j] = d*(Att[i][j]/ri[i]) + (1-d)/496
        else:
            H[i][j] = 1/ 496





matA = np.eye(496) - H.T
# create a row with 496's 1. 
newrow = np.array(np.repeat(1, len(Att)))

# replace first row of the raw data 

matA[0] = newrow


# create b with 496's 0
b_a = np.array(np.repeat(0, len(Att)))

# reshape to (496,1)
b_a = b_a.reshape(-1, 1)
# b.shape

# repalce 1 from the top

b_a[0] = np.array(1)


b_a.shape
# sparseAt =  sparse.csr_matrix(At)

(496, 1)

In [162]:
b_a

array([[1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [163]:
# forward substitution
def forward_substitution(L, b):
    
    #Get number of rows
    n = L.shape[0]
    
    #Allocating space for the solution vector
    y = np.zeros_like(b, dtype=np.double);
    
    #Here we perform the forward-substitution.  
    #Initializing  with the first row.
    y[0] = b[0] / L[0, 0]
    
    #Looping over rows in reverse (from the bottom  up),
    #starting with the second to last row, because  the 
    #last row solve was completed in the last step.
    for i in range(1, n):
        y[i] = (b[i] - np.dot(L[i,:i], y[:i])) / L[i,i]
        
    return y


# backward substituion
def back_substitution(U, y):
    
    #Number of rows
    n = U.shape[0]
    
    #Allocating space for the solution vector
    x = np.zeros_like(y, dtype=np.double);

    #Here we perform the back-substitution.  
    #Initializing with the last row.
    x[-1] = y[-1] / U[-1, -1]
    
    #Looping over rows in reverse (from the bottom up), 
    #starting with the second to last row, because the 
    #last row solve was completed in the last step.
    for i in range(n-2, -1, -1):
        x[i] = (y[i] - np.dot(U[i,i:], x[i:])) / U[i,i]
        
    return x

In [175]:
U, D, VT = svd(matA)

Dd = np.diag(D)

In [177]:
#U * D * VT * x = b_a

#D*VT*x = U'*b_a

w = forward_substitution(U, b_a)

Ddx = np.dot(VT.T, w)

x = np.dot(np.linalg.inv(Dd), Ddx)

In [178]:
for i in range(20):
    print(np.where(x == sorted(x, reverse=True)[i]))

(array([404], dtype=int64), array([0], dtype=int64))
(array([178], dtype=int64), array([0], dtype=int64))
(array([415], dtype=int64), array([0], dtype=int64))
(array([208], dtype=int64), array([0], dtype=int64))
(array([444], dtype=int64), array([0], dtype=int64))
(array([442], dtype=int64), array([0], dtype=int64))
(array([436], dtype=int64), array([0], dtype=int64))
(array([432], dtype=int64), array([0], dtype=int64))
(array([430], dtype=int64), array([0], dtype=int64))
(array([429], dtype=int64), array([0], dtype=int64))
(array([428], dtype=int64), array([0], dtype=int64))
(array([427], dtype=int64), array([0], dtype=int64))
(array([426], dtype=int64), array([0], dtype=int64))
(array([425], dtype=int64), array([0], dtype=int64))
(array([421], dtype=int64), array([0], dtype=int64))
(array([431], dtype=int64), array([0], dtype=int64))
(array([417], dtype=int64), array([0], dtype=int64))
(array([416], dtype=int64), array([0], dtype=int64))
(array([414], dtype=int64), array([0], dtype=i

In [164]:
U, E, Vt = svd(matA)
Ed = np.diag(E)

Ed

array([[22.29952042,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  3.0915098 ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  2.74983091, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.14384255,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.0975124 ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.04329257]])

In [165]:
SVD = np.linalg.multi_dot([U.T, np.linalg.inv(Ed), Vt.T])

sol = np.dot(SVD, b_a)
sol

array([[ 5.88006529e-03],
       [-4.95368778e-03],
       [-1.16555884e-03],
       [ 1.50246657e-02],
       [ 3.45305770e-03],
       [ 9.38265499e-03],
       [ 1.18195392e-02],
       [-1.32539754e-03],
       [ 3.54949759e-03],
       [ 4.99299400e-03],
       [ 2.95809769e-03],
       [-3.59139444e-03],
       [ 1.98336817e-03],
       [ 1.91657410e-02],
       [-2.48260058e-03],
       [-1.08253149e-03],
       [ 3.65844206e-03],
       [-1.75636721e-03],
       [-1.87679964e-03],
       [ 2.32999324e-02],
       [-8.63520036e-04],
       [ 1.25939430e-02],
       [-6.93690451e-03],
       [ 2.17113687e-03],
       [-7.55759835e-03],
       [-6.14287210e-03],
       [-1.50709771e-03],
       [-2.38757137e-03],
       [-2.87290760e-05],
       [ 2.64642001e-03],
       [-3.46532339e-03],
       [ 8.41029968e-03],
       [-1.29040670e-02],
       [ 9.27937330e-03],
       [ 1.29279347e-02],
       [ 1.92020556e-02],
       [ 3.42699017e-03],
       [-1.16104043e-02],
       [ 3.2

In [168]:
for i in range(20):
    print(np.where(sol == sorted(sol, reverse=True)[i]))

(array([145], dtype=int64), array([0], dtype=int64))
(array([236], dtype=int64), array([0], dtype=int64))
(array([306], dtype=int64), array([0], dtype=int64))
(array([102], dtype=int64), array([0], dtype=int64))
(array([301], dtype=int64), array([0], dtype=int64))
(array([297], dtype=int64), array([0], dtype=int64))
(array([404], dtype=int64), array([0], dtype=int64))
(array([394], dtype=int64), array([0], dtype=int64))
(array([326], dtype=int64), array([0], dtype=int64))
(array([395], dtype=int64), array([0], dtype=int64))
(array([241], dtype=int64), array([0], dtype=int64))
(array([354], dtype=int64), array([0], dtype=int64))
(array([92], dtype=int64), array([0], dtype=int64))
(array([170], dtype=int64), array([0], dtype=int64))
(array([245], dtype=int64), array([0], dtype=int64))
(array([296], dtype=int64), array([0], dtype=int64))
(array([389], dtype=int64), array([0], dtype=int64))
(array([166], dtype=int64), array([0], dtype=int64))
(array([167], dtype=int64), array([0], dtype=in

In [188]:
def ls_svd(A, b):
    m, n = A.shape
    U, sigma, Vh = svd(A, full_matrices=False, lapack_driver='gesdd')
    w = (U.T @ b)/sigma
    
    return Vh.T @ w

In [190]:
U, Sg, Vh = svd(matA)

In [192]:
w  = (U.T @ b_a)/Sg

In [199]:
sol = Vh.T @ w

solx = np.dot(sol, b_a)

In [202]:
U, E, V = svd(matA)

In [211]:
w, v = np.linalg.eig(H)

w.shape

(496,)

### Checking rank

In [212]:
for i in range(20):
    print(np.where(w == sorted(w, reverse=True)[i]))

(array([0], dtype=int64),)
(array([7], dtype=int64),)
(array([6], dtype=int64),)
(array([8], dtype=int64),)
(array([5], dtype=int64),)
(array([1], dtype=int64),)
(array([2], dtype=int64),)
(array([4], dtype=int64),)
(array([12], dtype=int64),)
(array([13], dtype=int64),)
(array([54], dtype=int64),)
(array([38], dtype=int64),)
(array([14], dtype=int64),)
(array([15], dtype=int64),)
(array([16], dtype=int64),)
(array([17], dtype=int64),)
(array([39], dtype=int64),)
(array([40], dtype=int64),)
(array([41], dtype=int64),)
(array([42], dtype=int64),)


In [201]:
temp1 = []

for i in range(20):
    temp1.append(np.where(solx == sorted(solx, reverse=True)[i]))

temp2 = []

for i in range(20):
    temp2.append(temp1[i][0][0])

urls = []

for i in temp2:
    urls.append(Udf.iloc[i-1][0])

urls

['http://staff.ucdavis.edu/#footer',
 'http://ucdavis.edu/help/privacy-accessibility.html',
 'http://universityofcalifornia.edu/sites/default/files/ucal-fb-image.png',
 'http://www.ucdmc.ucdavis.edu/welcome/maps_directions',
 'http://ogp.me/ --><meta property=',
 'http://www.musclemilk.com/terms-and-conditions',
 'http://policy.ucop.edu/doc/7000611/IMT-1300',
 'http://xmlns.com/foaf/0.1',
 'http://graduate.universityofcalifornia.edu/admissions',
 'http://om-as.ucdavis.edu/WebWO/ or call Facilities Management Customer Support Center at 530-752-1655.</span>\\r\\n',
 'http://ucd.alumniplans.com/covid-19-insurance-information.html',
 'http://www.berkeley.edu',
 'http://ogp.me/ns/fb#',
 'http://100.ucla.edu',
 'http://admission.universityofcalifornia.edu',
 'http://ogp.me/ns# fb: http://ogp.me/ns/fb#',
 'http://ucsd.edu/about/contact.html',
 'http://www.ucdmc.ucdavis.edu/medicalcenter/advantages.html',
 'http://ns.adobe.com/xap/1.0/sType/ResourceRef#',
 'http://www.alumni.ucdavis.edu']